# Разработка быстрой системы индексации и полнотекстового поиска с помощью Whoosh: библиотека Pure-Python

<u> Постановка проблемы </u>. У вас есть 1 миллион текстовых файлов в каталоге, и ваше приложение должно обеспечить поиск текстовых запросов по всем файлам в течение нескольких секунд (скажем, ~ 1-2 секунды).

## Введение: Whoosh

Whoosh - быстрая, многофункциональная библиотека полнотекстового индексирования и поиска, реализованная на чистом Python. Программисты могут использовать его, чтобы легко добавлять функции поиска в свои приложения и веб-сайты.

Пакет Whoosh pypi можно просто установить с помощью pip:

```
pip install Whoosh
```

Для примера, продемонстрированного в этом посте, вы можете загрузить набор данных из 70 000 текстовых файлов, взятых из простых вики-статей из [здесь](https://drive.google.com/file/d/1iKqQqrasxqDfTYwPyRjddxoUlPW602jv/view ).

## Создание проиндексированных данных: Whoosh

Все ваши текстовые файлы легко проиндексировать с помощью Whoosh. Первоначально должна быть определена схема индекса. Схема определяет список полей, которые будут проиндексированы или сохранены для каждого текстового файла. Это похоже на то, как мы определяем это для базы данных. Поле - это фрагмент информации для каждого документа в индексе, например его заголовок или текстовое содержимое. Индексирование поля означает, что его можно искать, и оно также возвращается с результатами, если оно определено в схеме как аргумент (сохранен = True). Вам нужно создать схему только один раз при создании индекса.

Наконец, все текстовые документы добавляются в средство записи индекса в цикле. Документы индексируются согласно схеме и должны быть добавлены согласно схеме. Ниже приведена реализация Python для индексации всех текстовых документов каталога.

In [1]:
import os
from whoosh.index import create_in
from whoosh.fields import Schema, TEXT, ID
import sys
 
def createSearchableData(root):   
 
    '''
    Schema definition: title(name of file), path(as ID), content(indexed
    but not stored),textdata (stored text content)
    '''
    schema = Schema(title=TEXT(stored=True),path=ID(stored=True),\
              content=TEXT,textdata=TEXT(stored=True))
    if not os.path.exists("indexdir"):
        os.mkdir("indexdir")
 
    # Creating a index writer to add document as per schema
    ix = create_in("indexdir",schema)
    writer = ix.writer()
 
    filepaths = [os.path.join(root,i) for i in os.listdir(root)]
    for path in filepaths:
        fp = open(path,'r')
        text = fp.read()
        writer.add_document(title=path.split("\\")[1], path=path,\
          content=text,textdata=text)
        fp.close()
    writer.commit()
 
root = "corpus"
createSearchableData(root)

## Запросы к индексированным данным

Запросы к индексированным данным состоят из двух важных частей:

<u> <b> Строка запроса </b> </u>: она передается при поиске в индексированных данных. Строка запроса может представлять собой одно слово, одно предложение, которое должно быть точно сопоставлено, несколько слов с «И», несколько слов с «ИЛИ» и т.д. Например:

* Запрос: политика (возвращает результат, если слово встречается)

* Запрос: спорт ИЛИ игры ИЛИ играть (возвращает результат, если какая-либо из строк встречается)

* Запрос: альфа-бета-гамма (возвращает результат, если документ содержит все строки)

* Запрос: «альфа-бета-гамма» (возвращает результат, если все строки встречаются вместе в документе).

<u> <b> Оценка </b> </u>. Каждый документ ранжируется в соответствии с функцией оценки. Существует довольно много типов функций оценки, которые поддерживает Whoosh:

<ol>
    <li>
        <i><b>Частота</b></i>: он просто возвращает количество терминов, встречающихся в документе. Он не выполняет никакой нормализации или взвешивания.
    </li><li>
    <i><b>Оценка Tf-Idf</b></i>: возвращает оценку tf * idf каждого документа.
    </li><li>
    <i><b>Оценка BM25F</b></i>: это функция ранжирования по умолчанию, используемая Whoosh. BM отвечает за лучшее соответствие. Он основан на tf-idf вместе с множеством факторов, таких как длина документа в словах, средняя длина документов в коллекции. Он также имеет свободные параметры k = 1.2 и b = 0.75.
    </li><li>
    <i><b>Косинусная оценка</b></i>: это полезно для поиска документа, похожего на ваш поисковый запрос.
    </li>
</ol>

Есть и [другие](https://github.com/jerem/Whoosh/blob/master/src/whoosh/scoring.py) оценки. 

Ниже приведена реализация Python для поиска запроса в индексированной базе данных:

In [2]:
from whoosh.qparser import QueryParser
from whoosh import scoring
from whoosh.index import open_dir
 
ix = open_dir("indexdir")
 
# query_str is query string
# query_str = sys.argv[1]
query_str = "hello world"
# Top 'n' documents as result
# topN = int(sys.argv[2])
topN = 10
 
with ix.searcher(weighting=scoring.Frequency) as searcher:
    query = QueryParser("content", ix.schema).parse(query_str)
    results = searcher.search(query,limit=topN)
    res_len = len(results)
    real_top = min(topN,res_len)
    print('Топ {:d}/{:d}:\n'.format(real_top,res_len))
    for i in range(real_top):
        # print(results[i]['title'], str(results[i].score), results[i]['textdata'])
        print('{:>9} {:4.1f}'.format(results[i]['title'], results[i].score))

Топ 0/0:



## Глоссарий

Whoosh [документация](https://whoosh.readthedocs.io/en/latest/glossary.html).

## Примеры запросов


![](images/capture1.png)
![](images/capture2.png)
![](images/capture3.png)